In [1]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/input/mri-predictor/tensorflow2/f10.96/1/my_cnn_model.h5')


2024-04-24 02:22:35.142097: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 02:22:35.142227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 02:22:35.235913: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img = cv2.resize(img, (256, 256)) 
            img_array = np.array(img)
            folder = input_folder.split("/")[-1]
            output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
            np.save(output_file_path, img_array)

jpg_to_npy_in_folders("/kaggle/input/alzheimersdisease5classdatasetadni/Alzheimers-ADNI/train", "/kaggle/working/train")

In [3]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1101

In [4]:
ad = [i for i in files if "AD" in i]
cn = [i for i in files if "CN" in i]
emci = [i for i in files if "EMCI" in i]
lmci = [i for i in files if "LMCI" in i]
mci = [i for i in files if " MCI " in i]
print(len(ad), len(cn), len(emci), len(lmci), len(mci))

145 493 204 61 198


In [5]:
import pandas as pd
l = [[ad, 0], [cn, 1], [emci, 2], [lmci, 3], [mci, 4]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])


In [6]:
images = []
for i in df["Path"]:
    try:
        images.append(np.load(i))
    except:
        print(i)
images = np.array(images)

In [7]:
import pandas as pd
df1 = pd.read_csv("/kaggle/input/handwriting-data-to-detect-alzheimers-disease/data.csv")
df1["class"] = pd.Series([1 if i == 'P' else 0 for i in df1["class"]])
df1

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,1
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,1
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,1
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,1
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,id_170,2930,0.000010,241.736477,176.115957,1839,6439,0.253347,0.174663,208.926217,...,0.119152,0.020909,4.508709,2.233198,96,44545,1798.923336,247448.3108,80335,0
170,id_171,2140,0.000009,274.728964,234.495802,2053,8487,0.225537,0.174920,254.612383,...,0.174495,0.017640,4.685573,2.806888,84,37560,1725.619941,160664.6464,345835,0
171,id_172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0
172,id_173,1760,0.000008,289.518195,196.411138,1674,6946,0.518937,0.202613,242.964666,...,0.114472,0.017194,3.493815,2.510601,88,51675,1915.573488,128727.1241,83445,0


In [8]:
x = df1.sample(50)
a = x["class"].value_counts()


In [9]:
q = df.sample(100)

In [10]:
nonAlz = x[x["class"] == 0]
d = []
e = []
count = 0
for i in q.values:
    if count == a[0]:
        break
    if i[0] == 1:
        d.append(i[1])
        e.append(i[0])
        count += 1

nonAlz.loc[:, "path"] = pd.Series(d, index=nonAlz.index)
nonAlz.loc[:, "mri-class"] = pd.Series(e, index=nonAlz.index)
nonAlz

/tmp/ipykernel_24/687741414.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonAlz.loc[:, "path"] = pd.Series(d, index=nonAlz.index)
/tmp/ipykernel_24/687741414.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonAlz.loc[:, "mri-class"] = pd.Series(e, index=nonAlz.index)


,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class,path,mri-class
131,id_132,1730,0.000008,140.555482,135.862670,887,5565,0.266189,0.126133,138.209076,...,6.193558,3.670072,96,25005,1908.840032,107715.63490,48485,0,/kaggle/working/train/Final CN JPEG410.npy,1
136,id_137,9375,0.000011,261.454359,232.595747,1613,8994,0.589141,0.168101,247.025053,...,8.013092,3.635344,65,27390,1821.010223,141386.35840,41160,0,/kaggle/working/train/Final CN JPEG216.npy,1
171,id_172,3830,0.000008,151.536989,171.104693,1287,7352,0.165480,0.161058,161.320841,...,3.493815,2.510601,88,51675,1915.573488,128727.12410,83445,0,/kaggle/working/train/Final CN JPEG79.npy,1
115,id_116,2785,0.000011,187.664373,152.993656,1357,7679,0.194407,0.154441,170.329015,...,7.012119,3.190663,64,31395,1827.406753,122689.62590,47190,0,/kaggle/working/train/Final CN JPEG249.npy,1
108,id_109,1755,0.000012,413.554928,219.636991,1692,9074,0.272219,0.169035,316.595959,...,4.699902,2.121379,83,47370,1275.107241,196117.54640,400760,0,/kaggle/working/train/Final CN JPEG296.npy,1
102,id_103,1350,0.000010,412.184223,242.281830,2536,6504,0.186999,0.198020,327.233027,...,5.891331,3.679741,96,36985,1523.410166,156158.38820,63100,0,/kaggle/working/train/Final CN JPEG241.npy,1
150,id_151,65,0.000006,875.811029,278.542889,2224,4606,0.949719,0.225705,577.176959,...,4.963550,5.078727,128,39750,1782.381006,210949.70930,87250,0,/kaggle/working/train/Final CN JPEG143.npy,1
129,id_130,2805,0.000008,135.318129,125.990855,1035,5310,0.322051,0.144647,130.654492,...,3.329704,2.008075,60,39775,1561.145820,147074.63340,64825,0,/kaggle/working/train/Final CN JPEG279.npy,1
127,id_128,1870,0.000006,595.572014,498.890817,2316,8467,1.043333,0.313464,547.231416,...,3.709496,2.166313,124,38330,1739.271067,264041.37160,77215,0,/kaggle/working/train/Final CN JPEG349.npy,1
101,id_102,96686,0.000006,358.184944,150.069558,1439,4351,1.553591,0.139658,254.127251,...,4.512446,2.767362,56,36095,1489.562820,109525.67730,55035,0,/kaggle/working/train/Final CN JPEG405.npy,1


In [11]:
Alz = x[x["class"] == 1]
d = []
e = []
count = 0
for i in q.values:
    if count == a[1]:
        break
    if i[0] != 1:
        d.append(i[1])
        e.append(i[0])
        count += 1

Alz.loc[:, "path"] = pd.Series(d, index=Alz.index)
Alz.loc[:, "mri-class"] = pd.Series(e, index=Alz.index)

Alz

/tmp/ipykernel_24/216200939.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alz.loc[:, "path"] = pd.Series(d, index=Alz.index)
/tmp/ipykernel_24/216200939.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Alz.loc[:, "mri-class"] = pd.Series(e, index=Alz.index)


,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class,path,mri-class
74,id_75,4975,0.000010,159.918971,144.289567,1450,8230,0.279051,0.135371,152.104269,...,4.021743,2.235643,110,69935,1956.559591,103355.56600,3302865,1,/kaggle/working/train/Final MCI JPEG111.npy,4
64,id_65,1835,0.000005,286.288141,357.869639,1857,4566,0.252665,0.290164,322.078890,...,2.780021,1.971110,168,64290,1359.878130,260234.32120,375875,1,/kaggle/working/train/Final EMCI JPEG202.npy,2
54,id_55,21800,0.000019,65.234661,72.407530,2931,8878,0.208208,0.164452,68.821096,...,4.316327,2.991742,129,47705,1764.116654,249158.72250,154130,1,/kaggle/working/train/Final MCI JPEG59.npy,4
82,id_83,1000,0.000007,396.923937,182.373694,1222,8282,0.319493,0.138608,289.648816,...,4.806361,3.335828,60,35920,1929.296214,94373.02807,59175,1,/kaggle/working/train/Final EMCI JPEG115.npy,2
88,id_89,910,0.000007,556.210702,160.192078,1879,6862,0.219201,0.136123,358.201390,...,4.806361,3.335828,60,35920,1929.296214,94373.02807,59175,1,/kaggle/working/train/Final LMCI JPEG53.npy,3
31,id_32,5100,0.000011,177.963556,237.609298,1648,6681,0.189135,0.186363,207.786427,...,2.594690,1.431078,171,131610,1663.374212,302207.89600,314885,1,/kaggle/working/train/Final EMCI JPEG30.npy,2
65,id_66,440,0.000009,1168.328276,168.852242,2839,7786,0.328469,0.163690,668.590259,...,4.211287,2.603010,62,30030,1643.603730,142390.12470,51210,1,/kaggle/working/train/Final MCI JPEG189.npy,4
29,id_30,9880,0.000015,137.524031,56.248264,901,7768,0.368335,0.181916,96.886147,...,5.662866,3.432212,71,38285,1705.952331,157649.09800,90485,1,/kaggle/working/train/Final EMCI JPEG28.npy,2
36,id_37,12960,0.000019,154.094876,66.137920,1335,9809,0.303533,0.168974,110.116398,...,5.171081,2.851422,81,33010,1583.533172,165641.91900,96545,1,/kaggle/working/train/Final AD JPEG140.npy,0
73,id_74,4515,0.000016,251.547355,210.799034,1940,15783,0.212121,0.168594,231.173194,...,2.940609,2.009832,112,56910,1792.763223,161309.83170,147180,1,/kaggle/working/train/Final AD JPEG142.npy,0


In [12]:
import pickle

with open('/kaggle/input/handwriitng-classifier/other/claassifier-gb/1/gb_classifier_weights.pkl', 'rb') as file:
    handwriting_model = pickle.load(file)

In [13]:
import random
s = [nonAlz, Alz]
x = []
for i in range(30):
    r = random.choice(s)
    x.append(r.sample(1))

In [14]:
def predictor(data):
    inp = data.drop(["ID", "class", "path", "mri-class"], axis=1)
    handwriting_prediction = handwriting_model.predict(inp)
    if handwriting_prediction.any() == 1:
        img = np.load(data["path"].values[0])
        img = np.expand_dims(img, axis=0) 
        prediction = np.argmax(model.predict(img), axis=1)[0]
        actual_case = f"have alzheimer's, type {data['mri-class'].values[0]}" if data["class"].any() == 1 else "you dont have alzheimer's"
        print(f"Model predicted - You have alzheimer's, type {prediction}")
        print(f"Actual case - You {actual_case}")
    else:
        print("Model predicted - You dont have alzheimer's")
        print(f"Actual case - You {'have' if data['class'].any() == 1 else 'dont have'} alzheimer's")

In [15]:
for i in x:
    predictor(i)

Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Model predicted - You have alzheimer's, type 1
Actual case - You have alzheimer's, type 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Model predicted - You have alzheimer's, type 4
Actual case - You have alzheimer's, type 4
Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


I0000 00:00:1713925381.795111      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Model predicted - You have alzheimer's, type 2
Actual case - You have alzheimer's, type 2
Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Model predicted - You have alzheimer's, type 2
Actual case - You have alzheimer's, type 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Model predicted - You have alzheimer's, type 4
Actual case - You have alzheimer's, type 4
Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Model predicted - You have alzheimer's, type 0
Actual case - You have alzheimer's, type 0
Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
Model predicted - You dont have alzheimer's
Actual case - You dont have alzheimer's
1/1 ━━━━━━━━━━━━━━━━━━